In [13]:
import torch
import intel_extension_for_pytorch as ipex
from sam2.sam2_image_predictor import SAM2ImagePredictor
import minio
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import urllib.request

In [14]:
def show_mask(mask, ax, obj_id=None, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        cmap = plt.get_cmap("tab10")
        cmap_idx = 0 if obj_id is None else obj_id
        color = np.array([*cmap(cmap_idx)[:3], 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)


def show_points(coords, labels, ax, marker_size=100):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)


def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0, 0, 0, 0), lw=2))

In [15]:
minio_client = minio.Minio(
    endpoint = "kazeserver.com:9000",
    access_key = "gC6Yek146dFGgmJRtyPK",
    secret_key = "45yleAnaiAUwt0X9McVwG9wZm7psD40BigMNz4Ds",
    secure = False
)

def get_minio_url(name: str):
    url = minio_client.presigned_get_object("highjump", name)
    return url

result = minio_client.list_objects('highjump', recursive=True, prefix='raw_data')
urls = [i for i in result]
url = get_minio_url(urls[387].object_name)


In [9]:
urllib.request.urlretrieve(url, 'test.mp4')

('test.mp4', <http.client.HTTPMessage at 0x7f6d09524610>)

In [10]:
def onclick(event):
    ix, iy = event.xdata, event.ydata
    points = np.array([[ix,iy]], dtype=np.float32)
    labels = np.array([1, 1], np.int32)
    print(ix,iy)
    show_points(points, labels, plt.gca())

video = "./test.mp4"

In [16]:
predictor = SAM2ImagePredictor.from_pretrained("facebook/sam2-hiera-large", device="cpu")


/home/kazewong/Environment/SKO/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-20 17:35:21,860 - root - INFO - Loaded checkpoint sucessfully


In [12]:
checkpoint = "../models/sam2.1_hiera_tiny.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_t.yaml"
predictor = build_sam2_video_predictor(model_cfg, checkpoint)
with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    state = predictor.init_state(video_path=video)
    predictor.reset_state(state)

2025-01-20 17:09:46,422 - root - INFO - Loaded checkpoint sucessfully


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
ann_frame_idx = 150  # the frame index we interact with
ann_obj_id = 1  # give a unique id to each object we interact with (it can be any integers)
points = np.array([[1250, 600],  [1300,680]], dtype=np.float32)
# for labels, `1` means positive click and `0` means negative click
labels = np.array([1,0], np.int32)
_, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
    inference_state=state,
    frame_idx=ann_frame_idx,
    obj_id=ann_obj_id,
    points=points,
    labels=labels,
)
plt.figure(figsize=(9, 6))
plt.title(f"frame {ann_frame_idx}")
plt.imshow(Image.open(os.path.join(video_dir, frame_names[ann_frame_idx])))
show_points(points, labels, plt.gca())
show_mask((out_mask_logits[0] > 0.0).cpu().numpy(), plt.gca(), obj_id=out_obj_ids[0])

In [ ]:
ann_frame_idx = 150  # the frame index we interact with
ann_obj_id = 2  # give a unique id to each object we interact with (it can be any integers)
points = np.array([[1300, 700], [1300,680]], dtype=np.float32)
# for labels, `1` means positive click and `0` means negative click
labels = np.array([1, 1], np.int32)
_, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
    inference_state=state,
    frame_idx=ann_frame_idx,
    obj_id=ann_obj_id,
    points=points,
    labels=labels,
)
plt.figure(figsize=(12, 8))
plt.title(f"frame {ann_frame_idx}")
plt.imshow(Image.open(os.path.join(video_dir, frame_names[ann_frame_idx])))
show_points(points, labels, plt.gca())
show_mask((out_mask_logits[0] > 0.0).cpu().numpy(), plt.gca(), obj_id=out_obj_ids[0])
show_mask((out_mask_logits[1] > 0.0).cpu().numpy(), plt.gca(), obj_id=out_obj_ids[1])

In [ ]:
ann_frame_idx = 150  # the frame index we interact with
ann_obj_id = 3  # give a unique id to each object we interact with (it can be any integers)
points = np.array([[1350, 750], [1350,740]], dtype=np.float32)
# for labels, `1` means positive click and `0` means negative click
labels = np.array([1, 1], np.int32)
_, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
    inference_state=state,
    frame_idx=ann_frame_idx,
    obj_id=ann_obj_id,
    points=points,
    labels=labels,
)
ann_obj_id = 4  # give a unique id to each object we interact with (it can be any integers)
points = np.array([[1310, 770], [1320,780]], dtype=np.float32)
# for labels, `1` means positive click and `0` means negative click
labels = np.array([1, 1], np.int32)
_, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
    inference_state=state,
    frame_idx=ann_frame_idx,
    obj_id=ann_obj_id,
    points=points,
    labels=labels,
)



In [ ]:
plt.figure(figsize=(12, 8))
plt.title(f"frame {ann_frame_idx}")
plt.imshow(Image.open(os.path.join(video_dir, frame_names[ann_frame_idx])))
show_points(points, labels, plt.gca())
for i in range(len(out_obj_ids)):
    show_mask((out_mask_logits[i] > 0.0).cpu().numpy(), plt.gca(), obj_id=out_obj_ids[i])


In [ ]:
# run propagation throughout the video and collect the results in a dict
video_segments = {}  # video_segments contains the per-frame segmentation results
with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
    for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(state, start_frame_idx=0):
        video_segments[out_frame_idx] = {
            out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
            for i, out_obj_id in enumerate(out_obj_ids)
        }


In [ ]:

# render the segmentation results every few frames
vis_frame_stride = 50
plt.close("all")
for out_frame_idx in range(0, len(frame_names), vis_frame_stride):
    plt.figure(figsize=(12, 8))
    plt.title(f"frame {out_frame_idx}")
    plt.imshow(Image.open(os.path.join(video_dir, frame_names[out_frame_idx])))
    for out_obj_id, out_mask in video_segments[out_frame_idx].items():
        show_mask(out_mask, plt.gca(), obj_id=out_obj_id)